<a href="https://colab.research.google.com/github/Maagnitude/diabetes-xai/blob/main/diabetes_xai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install shap

In [ ]:
# Import SHAP and XGBoost method libraries
import numpy as np
import xgboost
import shap

In [ ]:
#Import diabetes dataset
ds_X, ds_Y = shap.datasets.diabetes()

Τυπώνουμε τα πρώτα 5 δείγματα του dataset, που το έχουμε σε μορφή DataFrame, για να ελέγχξουμε τη αριθμητική μορφή των δεδομένων μας.

In [ ]:
ds_X.head()

Δεν έχει null values, και όλες οι τιμές του είναι τύπου float.

In [ ]:
ds_X.info()

In [ ]:
# X_train,X_test,y_train,y_test = train_test_split(ds_X, ds_Y, test_size=0.2, random_state=0)

In [ ]:
#Train an XGBoost regression model
model = xgboost.XGBRegressor().fit(ds_X, ds_Y)

#Estimate the SHAP values for each dataset sample
explainer = shap.Explainer(model)
shap_values = explainer(ds_X)

## **EXPLANATIONS**

# **D. Γράφημα τιμών Shapley για τυχαία επιλεγμένο δείγμα.**
Στο παρακάτω γράφημα τιμών Shapley (**διάγραμμα καταρράκτη**) για το τυχαία επιλεγμένο δείγμα μας (νο.345), έχουμε στον κάθετο άξονα τα χαρακτηριστικά μας, ιεραρχημένα ως προς την σημαντικότητα τους, μαζί με τις τιμές Shapley, και στον οριζόντιο άξονα έχουμε τις τιμές που μπορεί να έχει η πρόβλεψη. Οι παρατηρήσεις που κάνουμε είναι οι εξής:
* Αρχικά, το χαρακτηριστικό με την μεγαλύτερη **αρνητική** συνεισφορά είναι το **bmi** (**-7.2**), και με τη μεγαλύτερη **θετική** συνεισφορά είναι το **age** (**+5.51**). Από αυτό καταλαβαίνουμε ότι για κάθε μονάδα **αύξησης** του **bmi**, η συνολική πρόβλεψη του δείγματος **πέφτει** κατά **7.2**, και για κάθε μονάδα **αύξησης** του **age**, **ανεβαίνει** η συνολική πρόβλεψη κατά **5.51** (στο **συγκεκριμένο δείγμα** πάντα)
* Το άθροισμα των συνεισφορών ισούται με την διαφορά της πρόβλεψης του δείγματος και της μέσης πρόβλεψης, δηλ. **f(x) - E[f(x)] = 132.084 - 152.088 = -20.004**, μιας και η συνεισφορά τους είναι αυτή που οδηγεί την πρόβλεψη του δείγματος πιο πάνω ή πιο κάτω από την μέση πρόβλεψη. Όντως αν προσθέσουμε όλες τις τιμές των συνεισφορών, θα έχουμε -20.004 (λόγω στρογγυλοποιημένων τιμών στο διάγραμμα βγαίνει 19.98.)
* Κάθε αρνητική τιμή (μπλε βέλος προς τα αριστερά) αφορά χαρακτηριστικό με αρνητική συνεισφορά και κάθε θετική τιμή (κόκκινο βέλος προς τα δεξιά) αφορά χαρακτηριστικό με θετική συνεισφορά.

In [ ]:
from statistics import mean

# Προσδιορίζουμε το τυχαίο δείγμα που θα εξετάσουμε (local explanations)
our_rand_indx = 345

# Υπολογισμός προβλέψεων μοντέλου
predictions = model.predict(ds_X)

average_prediction = mean(predictions)
actual_prediction = predictions[our_rand_indx]
difference = actual_prediction - average_prediction

print(f'Actual Prediction: {actual_prediction}')
print(f'Average Prediction: {average_prediction}')
print(f'Difference: {difference}')

# Οπτικοποιούμε τις τιμές Shapley για το δείγμα μας
# για την διερεύνηση των συνεισφορών κάθε χαρακτηριστικού.
shap.plots.waterfall(shap_values[our_rand_indx])

# **E. Tο γράφημα δύναμης (βάσει τιμών Shapley) για τυχαία επιλεγμένο δείγμα**
Στο παρακάτω **γράφημα δύναμης**, παρατηρούμε μια ομοιότητα με το **διάγραμμα καταρράκτη**. Μπορούμε λοιπόν για το δείγμα μας να πάρουμε τις ίδιες πληροφορίες. Η **ουσιαστική διαφορά** είναι ότι έχει πάρει τις **θετικές-αρνητικές συνεισφορές**, τις έχει ευθυγραμμίσει και έχει βάλει **αριστερά** τις **θετικές** και **δεξιά** τις **αρνητικές**. Το σημείο που ενώνονται είναι η **συνολική πρόβλεψη του δείγματος** (**f(x) = 132.08**).

Το **base value** είναι η μέση τιμή πρόβλεψης όλων των δειγμάτων, δηλαδή το **E[f(x)]** που είδαμε στο παραπάνω διάγραμμα ότι είναι **152.088**

In [ ]:
# Οπτικοποιούμε (SHAP-based) το γράφημα δύναμης για το δείγμα μας. 
# (local_explanation)
shap.initjs()
shap.plots.force(shap_values[our_rand_indx], matplotlib=True)

# **F. Γράφημα περίληψης (βάσει τιμών Shapley) για το σύνολο των δειγμάτων**
Στο παρακάτω **διάγραμμα περίληψης**, έχουμε αριστερά **ιεραρχημένα** τα εξεταζόμενα χαρακτηριστικά των δειγμάτων μας ανάλογα με την **σπουδαιότητα** τους, δεξιά μία μπάρα (**High-Low**) που αφορά τις τιμές των χαρακτηριστικών, και στον οριζόντιο άξονα έχουμε τις τιμές **Shapley**. Ύστερα κάνουμε τις εξής παρατηρήσεις για το σύνολο των δειγμάτων μας:
* **Χαμηλή** τιμή του χαρακτηριστικού **s5**, **μειώνει** τις τιμές των ποσοτικών μετρήσεων που αφορούν τον **Διαβήτη**, ενώ υψηλή τιμή αυτού (**s5**), αυξάνουν τις τιμές των ποσοτικών αυτών μετρήσεων. Το ίδιο ισχύει σε λίγο **μικρότερο βαθμό** και για τις τιμές των χαρακτηριστικών **bmi** και **bp**.
* Σε **αντίθετη** περίπτωση, **χαμηλή** τιμή των χαρακτηριστικών **sex** και **s3**, αυξάνει τις τιμές των ποσοτικών μετρήσεων που αφορούν τον Διαβήτη, ενώ υψηλή τιμή αυτών, μειώνουν τις τιμές των ποσοτικών μετρήσεων. Το ίδιο, σε μικρότερο βαθμό, και για τα χαρακτηριστικά **s2** και **s1** με αρκετά, βέβαια, **outliers** (μεμονωμένες τελείες στα άκρα των τιμών **Shapley**).
* 

In [ ]:
# Οπτικοποίηση (SHAP-based) με διάγραμμα περίληψης όλων των χαρακτηριστικών
# για όλα τα δείγματα του συνόλου δεδομένων (global explanation)
shap.initjs()
shap.plots.beeswarm(shap_values)

# **G. Γράφημα μέσης απόλυτης τιμής των τιμών Shapley για το σύνολο των δειγμάτων**
Στο παρακάτω διάγραμμα, οπτικοποιούμε την σπουδαιότητα των χαρακτηριστικών, τα οποία έχουμε στον κάθετο άξονα, ιεραρχιμένα βάσει της σπουδαιότητας αυτής (με **φθίνουσα σημασία**). Στον οριζόντιο άξονα έχουμε τη μέση απόλυτη τιμή των τιμών **Shapley** των χαρακτηριστικών, για κάθε δείγμα από το σύνολο των δειγμάτων μας. Επειδή το διάγραμμα αφορά τη μέση απόλυτη τιμή, έχουμε μόνο θετικές τιμές συνεισφοράς, για λόγους σύγκρισης. Κάνουμε, λοιπόν, τις εξής παρατηρήσεις:
* Το χαρακτηριστικό με την μεγαλύτερη συνεισφορά είναι το **s5** (**+28.89**), και το αμέσως επόμενο είναι το **bmi** (**+22.25**). Αυτό που καταλαβαίνουμε είναι ότι μια μονάδα **μεταβολής** στο χαρακτηριστικό **s5**, μεταβάλλει κατά μέσο όρο **28.89** μονάδες στην συνολική πρόβλεψη.
* Το χαρακτηριστικό με την μικρότερη συνεισφορά είναι το **s4** (**+1.37**) και το αμέσως επόμενο το **s1** (**+2.33**). Δηλαδή η **μεταβολή** τους **δεν συνεισφέρουν** ιδιαίτερα στην μεταβολή της πρόβλεψης.

In [ ]:
# Οπτικοποίηση της μέσης απόλυτης τιμής των τιμών Shapley για κάθε
# χαρακτηριστικό για όλα τα δείγματα του συνόλου δεδομένων (global explanation)
shap.initjs()
shap.plots.bar(shap_values)

# **H. Διαδραστικό γράφημα δύναμης για το σύνολο των δειγμάτων (δώστε ενδεικτικά παραδείγματα/εξηγήσεις/παρατηρήσεις από την εξέταση του γραφήματος)**
Στο διαδραστικό αυτό διάγραμμα έχουμε ουσιαστικά ενωμένα τα **διαγράμματα δύναμης** όλων των δειγμάτων (απλά τα έχουμε **κάθετα**, σε αντίθεση με το διάγραμμα στο **ζήτημα Ε** όπου το είχαμε **οριζόντιο**). Στον κάθετο άξονα έχουμε τις τιμές των παραγόμενων προβλέψεων και στον οριζόντιο άξονα έχουμε τα δείγματα ταξινομημένα με τον τρόπο που έχουμε ορίσει στο drop down menu. Στην περίπτωση μας το έχουμε ταξινομημένα με βάση την ομοιότητα τους "**sample order by similarity**". 
* Συγκεκριμένα παρατηρούμε ότι **αριστερά** βρίσκονται δείγματα των οποίων τα χαρακτηριστικά συνεισφέρουν κατά μεγάλο ποσοστό **θετικά**, με αποτέλεσμα πιο αριστερά να βρίσκονται δείγματα με συγκριτικά **μεγάλες τιμές πρόβλεψης**.
* Αντίθετα, προς τα δεξία, κυριαρχούν οι **αρνητικές** συνεισφορές, με αποτέλεσμα να βρίσκονται τα δείγματα με συγκριτικά **μικρές τιμές** συνολικής πρόβλεψης.
* Επίσης, παρατηρούμε ότι στα **αριστερά**  έχουμε σχηματισμό **ομάδας δειγμάτων με υψηλές προβλέψεις** (δηλ. πιθανές υψηλές ποσοτικές μετρήσεις για την νόσο του διαβήτη. 


In [ ]:
# Οπτικοποίηση με χρήση διαδραστικού διαγράμματος δύναμης για όλα τα δείγματα
# του συνόλου δεδομένων (global explanation)
shap.initjs()
shap.plots.force(shap_values)